In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from random import randint
from time import sleep
from itertools import islice
from pandas import json_normalize

In [2]:
secrets_file = open("secrets.txt","r")

In [3]:
string = secrets_file.read()
#string

In [27]:
#string.split('\n')

In [5]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
#secrets_dict

In [6]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [ ]:
# Getting the playlist
playlist = sp.user_playlist_tracks("spotify", "37i9dQZEVXbNM8vS9cIqAG")
#playlist

In [8]:
def get_playlist_tracks(playlist_id):
    """
    Retrieves all tracks from a Spotify playlist.

    Args:
        playlist_id (str): The unique identifier of the playlist.

    Returns:
        list: A list of dictionaries, where each dictionary represents a track in the playlist.
    """
    results = sp.user_playlist_tracks("spotify", playlist_id)
    tracks = results['items']
    while results['next'] is not None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1, 3))  # Respectful nap
        
    return tracks

In [9]:
playlist = get_playlist_tracks('37i9dQZEVXbNM8vS9cIqAG')
len(playlist)

50

In [10]:
def get_name_artists_from_track(track):
    """
    Extracts the names of artists from a track dictionary.

    Args:
        track (dict): A dictionary representing a music track, including artist details.

    Returns:
        list: A list of artist names associated with the track.
    """
    
    return [artist["name"] for artist in track["artists"]]

In [11]:
# Same for artist id
def get_name_artists_id_from_track(track):
    return [artist["id"] for artist in track["artists"]]

In [12]:
def get_tracks(playlist):
    """
    Extracts detailed information from a list of track dictionaries in a playlist.

    Args:
        playlist (list): A list of dictionaries, where each dictionary represents a music track.

    Returns:
        list: A list of lists, where each inner list represents detailed information about a track.
    """
    tracklist = []
    for i in range(len(playlist)):
        # Extract specific track details and append them to the tracklist
        # (Include details like URI, name, artist, album, duration, explicitness, and popularity)
        tracklist.append([
            playlist[i]['track']['uri'],
            playlist[i]['track']['name'],
            get_name_artists_from_track(playlist[i]['track'])[0],
            get_name_artists_id_from_track(playlist[i]['track'])[0],
            playlist[i]['track']['album']['id'],
            playlist[i]['track']['album']['name'],
            playlist[i]['track']['duration_ms'],
            playlist[i]['track']['explicit'],
            playlist[i]['track']['popularity']
        ])
        
    return tracklist

In [13]:
songs = get_tracks(playlist)

In [14]:
len(songs)

50

In [15]:
df_songs = pd.DataFrame(data = songs, columns = ['uri','title','artist_name','artist_id','album_id','album_name','length','explicit','popularity'])

In [16]:
df_songs.head()

,uri,title,artist_name,artist_id,album_id,album_name,length,explicit,popularity
0,spotify:track:05DrM4jF73TZFu1h7sl26w,DELLALI (feat. Hamza),ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,33ibPX6kKMGWXcxNtWYlv7,DELLALI (feat. Hamza),182037,True,71
1,spotify:track:0e6eVGeec6asBnmE4gTYqA,3DABI,Draganov,2g8Pu5gVtDpkYGsP3RLepJ,08ZFlBocogt3UFTyEN7Ob4,3DABI,202000,True,69
2,spotify:track:6Yu03XnLWHxvBPbu7tzYBe,WELD LAADOUL,ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,6K7uE5ShRxDxz9UWFMxSLx,WELD LAADOUL,218937,True,65
3,spotify:track:6JI15EdgbGuhUjm4tWyIco,SEYA,GIMS,0GOx72r5AAEKRGQFn3xqXK,3zIRFFMBelYsHN0ksstyWU,LES DERNIÈRES VOLONTÉS DE MOZART (SYMPHONY),188484,False,70
4,spotify:track:5OInx13SpghEifbS5bQmwq,Dawk Lya,Snor,7LC7LQ1f1CzaRLobFcblZN,4vbm2Tj7L5fbzD1q5MmFrP,Dawk Lya,148062,True,64


In [17]:
df_songs.shape

(50, 9)

In [18]:
def get_features_delayed(uri):
    """
    Retrieves audio features of a Spotify track using its URI with a respectful delay.

    Args:
        uri (str): The URI of the Spotify track for which audio features are requested.

    Returns:
        dict: A dictionary containing audio features of the track.
    """
    sleep(randint(1, 2))  # A respectful nap (waits for 1-2 seconds)
    
    return sp.audio_features(uri)

In [19]:
# Running this takes too long

df_songs['features'] = df_songs['uri'].apply(get_features_delayed).copy()
df_songs

,uri,title,artist_name,artist_id,album_id,album_name,length,explicit,popularity,features
0,spotify:track:05DrM4jF73TZFu1h7sl26w,DELLALI (feat. Hamza),ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,33ibPX6kKMGWXcxNtWYlv7,DELLALI (feat. Hamza),182037,True,71,"[{'danceability': 0.572, 'energy': 0.565, 'key..."
1,spotify:track:0e6eVGeec6asBnmE4gTYqA,3DABI,Draganov,2g8Pu5gVtDpkYGsP3RLepJ,08ZFlBocogt3UFTyEN7Ob4,3DABI,202000,True,69,"[{'danceability': 0.744, 'energy': 0.394, 'key..."
2,spotify:track:6Yu03XnLWHxvBPbu7tzYBe,WELD LAADOUL,ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,6K7uE5ShRxDxz9UWFMxSLx,WELD LAADOUL,218937,True,65,"[{'danceability': 0.652, 'energy': 0.576, 'key..."
3,spotify:track:6JI15EdgbGuhUjm4tWyIco,SEYA,GIMS,0GOx72r5AAEKRGQFn3xqXK,3zIRFFMBelYsHN0ksstyWU,LES DERNIÈRES VOLONTÉS DE MOZART (SYMPHONY),188484,False,70,"[{'danceability': 0.798, 'energy': 0.681, 'key..."
4,spotify:track:5OInx13SpghEifbS5bQmwq,Dawk Lya,Snor,7LC7LQ1f1CzaRLobFcblZN,4vbm2Tj7L5fbzD1q5MmFrP,Dawk Lya,148062,True,64,"[{'danceability': 0.609, 'energy': 0.399, 'key..."
5,spotify:track:1bsoYydWFNmocIOD2eeWI5,Hyati,Duke,6JLJjTC6pSfJousfGwdn6r,4YwFCbGREEY27uNJ7nxTEN,Hyati,155280,False,63,"[{'danceability': 0.785, 'energy': 0.485, 'key..."
6,spotify:track:2Pn4RjpmuYRMJTXWzTonXn,ZIGZAG,Bo9al,0DF2Pcrc3dY80zqbmgKHCF,00AGiSuiFx3kmEIbj0ENeK,ZIGZAG,168042,True,62,"[{'danceability': 0.754, 'energy': 0.726, 'key..."
7,spotify:track:23iRPid7TsITfNxKGL0toM,KIF KIF,Lbenj,1H0D7p5aN8tGG8DPLt0Nbv,7DeGLtJG2R45QJrHW1NLLF,KIF KIF,227826,False,62,"[{'danceability': 0.544, 'energy': 0.726, 'key..."
8,spotify:track:3ejjmzEsUfObGdmmQRVmM2,Qui sait ? (feat. ElGrandeToto),Niro,1wAtSe79kItIb9nf5EhI2Q,2mcTAfultyCrXKx1IE1XQZ,Taulier,190786,True,69,"[{'danceability': 0.534, 'energy': 0.624, 'key..."
9,spotify:track:2N0SPREDYqILVEFSsWF5N5,Love Me Again,V,3JsHnjpbhX4SnySpvpa9DK,1ZBDI1LqnuAKdrePUF2D5m,Layover,182293,False,90,"[{'danceability': 0.633, 'energy': 0.607, 'key..."


In [20]:
print(get_features_delayed('spotify:track:6JI15EdgbGuhUjm4tWyIco'))

[{'danceability': 0.798, 'energy': 0.681, 'key': 10, 'loudness': -4.818, 'mode': 0, 'speechiness': 0.0722, 'acousticness': 0.219, 'instrumentalness': 0.00197, 'liveness': 0.137, 'valence': 0.434, 'tempo': 101.005, 'type': 'audio_features', 'id': '6JI15EdgbGuhUjm4tWyIco', 'uri': 'spotify:track:6JI15EdgbGuhUjm4tWyIco', 'track_href': 'https://api.spotify.com/v1/tracks/6JI15EdgbGuhUjm4tWyIco', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6JI15EdgbGuhUjm4tWyIco', 'duration_ms': 188485, 'time_signature': 4}]


In [21]:
# We have to make the function resilient to errors in case the coinnection breaks
def get_features_delayed(uri):
    # First we try to get the features
    try:
        sleep(randint(1,2)) # respectful nap
        features = sp.audio_features(uri)
    # If it doesnt work we try again, for this we make the function recursive
    except:
        print('Error occured while getting feature names')
        features = get_features_delayed(uri)
        
    # Finally we return the features
    return features

In [22]:
df_songs.head()

,uri,title,artist_name,artist_id,album_id,album_name,length,explicit,popularity,features
0,spotify:track:05DrM4jF73TZFu1h7sl26w,DELLALI (feat. Hamza),ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,33ibPX6kKMGWXcxNtWYlv7,DELLALI (feat. Hamza),182037,True,71,"[{'danceability': 0.572, 'energy': 0.565, 'key..."
1,spotify:track:0e6eVGeec6asBnmE4gTYqA,3DABI,Draganov,2g8Pu5gVtDpkYGsP3RLepJ,08ZFlBocogt3UFTyEN7Ob4,3DABI,202000,True,69,"[{'danceability': 0.744, 'energy': 0.394, 'key..."
2,spotify:track:6Yu03XnLWHxvBPbu7tzYBe,WELD LAADOUL,ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,6K7uE5ShRxDxz9UWFMxSLx,WELD LAADOUL,218937,True,65,"[{'danceability': 0.652, 'energy': 0.576, 'key..."
3,spotify:track:6JI15EdgbGuhUjm4tWyIco,SEYA,GIMS,0GOx72r5AAEKRGQFn3xqXK,3zIRFFMBelYsHN0ksstyWU,LES DERNIÈRES VOLONTÉS DE MOZART (SYMPHONY),188484,False,70,"[{'danceability': 0.798, 'energy': 0.681, 'key..."
4,spotify:track:5OInx13SpghEifbS5bQmwq,Dawk Lya,Snor,7LC7LQ1f1CzaRLobFcblZN,4vbm2Tj7L5fbzD1q5MmFrP,Dawk Lya,148062,True,64,"[{'danceability': 0.609, 'energy': 0.399, 'key..."


In [23]:
def flatten_features(df):
    """
    Flatten audio features data in a DataFrame.

    This function takes a DataFrame containing audio features and flattens the data
    by extracting specific audio feature values for each row and creating new columns
    in the DataFrame with these values.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'features' column.

    Returns:
        pd.DataFrame: A new DataFrame with flattened audio feature columns.
        
    """
    featurelist = []
    for i in range(len(df['features'])):
        try:
            featurelist.append([
            df['features'][i][0]['danceability'],
            df['features'][i][0]['energy'],
            df['features'][i][0]['key'],
            df['features'][i][0]['loudness'],
            df['features'][i][0]['mode'],
            df['features'][i][0]['speechiness'],
            df['features'][i][0]['acousticness'],
            df['features'][i][0]['instrumentalness'],
            df['features'][i][0]['liveness'],
            df['features'][i][0]['valence'],
            df['features'][i][0]['tempo']
            ])
        except:
            featurelist.append([0,0,0,0,0,0,0,0,0,0,0])
    featureframe = pd.DataFrame(featurelist,columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo'] )
    df = pd.concat([df,featureframe],axis = 1)
    df = df.drop('features',axis = 1)
    
    return df

In [24]:
# Testing the functions takes too long

df_songs2 = df_songs.copy()
df_test = flatten_features(df_songs)
df_test.head(5)

,uri,title,artist_name,artist_id,album_id,album_name,length,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:05DrM4jF73TZFu1h7sl26w,DELLALI (feat. Hamza),ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,33ibPX6kKMGWXcxNtWYlv7,DELLALI (feat. Hamza),182037,True,71,0.572,0.565,1,-7.664,0,0.1730,0.697,0.00000,0.1690,0.152,96.246
1,spotify:track:0e6eVGeec6asBnmE4gTYqA,3DABI,Draganov,2g8Pu5gVtDpkYGsP3RLepJ,08ZFlBocogt3UFTyEN7Ob4,3DABI,202000,True,69,0.744,0.394,8,-12.518,0,0.1600,0.198,0.00000,0.5490,0.465,136.882
2,spotify:track:6Yu03XnLWHxvBPbu7tzYBe,WELD LAADOUL,ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,6K7uE5ShRxDxz9UWFMxSLx,WELD LAADOUL,218937,True,65,0.652,0.576,7,-6.954,0,0.3750,0.570,0.00000,0.1230,0.358,91.740
3,spotify:track:6JI15EdgbGuhUjm4tWyIco,SEYA,GIMS,0GOx72r5AAEKRGQFn3xqXK,3zIRFFMBelYsHN0ksstyWU,LES DERNIÈRES VOLONTÉS DE MOZART (SYMPHONY),188484,False,70,0.798,0.681,10,-4.818,0,0.0722,0.219,0.00197,0.1370,0.434,101.005
4,spotify:track:5OInx13SpghEifbS5bQmwq,Dawk Lya,Snor,7LC7LQ1f1CzaRLobFcblZN,4vbm2Tj7L5fbzD1q5MmFrP,Dawk Lya,148062,True,64,0.609,0.399,0,-11.177,0,0.0362,0.443,0.06330,0.0783,0.360,149.946


In [25]:
def get_playlist_tracks_features_to_dataframe(playlist_id):
    """
    Retrieves tracks and their features from a Spotify playlist and returns them as a pandas DataFrame.

    Args:
    playlist_id (str): The Spotify ID of the playlist.

    Returns:
    pandas.DataFrame: A DataFrame containing track URI, title, artist name, artist ID, album ID, album name, length, explicit status, popularity, and audio features.
    """

    # Initialize the final DataFrame
    full_dataframe = pd.DataFrame()

    # Retrieve the first group of songs from the playlist
    results = sp.user_playlist_tracks("spotify", playlist_id)

    # Monitor progress
    errorcount = 0
    fetched = 100

    # Process the first batch of songs
    while results:
        try:
            # Extract data and convert it into a DataFrame
            flat = get_tracks(results['items'])
            resultframe = pd.DataFrame(data=flat, columns=['uri', 'title', 'artist_name', 'artist_id', 'album_id', 'album_name', 'length', 'explicit', 'popularity'])
            
            # Retrieve features of the songs
            resultframe['features'] = resultframe['uri'].apply(get_features_delayed)
            
            # Flatten the features and add the results to the final DataFrame
            resultframe = flatten_features(resultframe)
            full_dataframe = full_dataframe.append(resultframe, ignore_index=True)
            
            # Move to the next batch of songs, if available
            results = sp.next(results) if results['next'] else None

            # Report progress
            fetched += len(resultframe)
            print(f'Fetched {fetched} out of {results["total"]} ({100 * (fetched / results["total"])}%)' if results else "Fetch complete.")

        except Exception as e:
            errorcount += 1
            print(f'Error while fetching. # {errorcount}. Error: {e}')

    return full_dataframe

In [26]:
# Call the function with the playlist ID
small_playlist_id = '37i9dQZEVXbNM8vS9cIqAG'
test_df = get_playlist_tracks_features_to_dataframe(small_playlist_id)

# Display the head of the resulting DataFrame
test_df.head()

Fetch complete.


/var/folders/09/qkspll5d37l0g2941hv9rvs80000gn/T/ipykernel_1126/109683254.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_dataframe = full_dataframe.append(resultframe, ignore_index=True)


,uri,title,artist_name,artist_id,album_id,album_name,length,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:05DrM4jF73TZFu1h7sl26w,DELLALI (feat. Hamza),ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,33ibPX6kKMGWXcxNtWYlv7,DELLALI (feat. Hamza),182037,True,71,0.572,0.565,1,-7.664,0,0.1730,0.697,0.00000,0.1690,0.152,96.246
1,spotify:track:0e6eVGeec6asBnmE4gTYqA,3DABI,Draganov,2g8Pu5gVtDpkYGsP3RLepJ,08ZFlBocogt3UFTyEN7Ob4,3DABI,202000,True,69,0.744,0.394,8,-12.518,0,0.1600,0.198,0.00000,0.5490,0.465,136.882
2,spotify:track:6Yu03XnLWHxvBPbu7tzYBe,WELD LAADOUL,ElGrandeToto,4BFLElxtBEdsdwGA1kHTsx,6K7uE5ShRxDxz9UWFMxSLx,WELD LAADOUL,218937,True,65,0.652,0.576,7,-6.954,0,0.3750,0.570,0.00000,0.1230,0.358,91.740
3,spotify:track:6JI15EdgbGuhUjm4tWyIco,SEYA,GIMS,0GOx72r5AAEKRGQFn3xqXK,3zIRFFMBelYsHN0ksstyWU,LES DERNIÈRES VOLONTÉS DE MOZART (SYMPHONY),188484,False,70,0.798,0.681,10,-4.818,0,0.0722,0.219,0.00197,0.1370,0.434,101.005
4,spotify:track:5OInx13SpghEifbS5bQmwq,Dawk Lya,Snor,7LC7LQ1f1CzaRLobFcblZN,4vbm2Tj7L5fbzD1q5MmFrP,Dawk Lya,148062,True,64,0.609,0.399,0,-11.177,0,0.0362,0.443,0.06330,0.0783,0.360,149.946
